In [2]:
!pip install langchain --quiet
!pip install google-search-results --quiet

In [3]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler

parameters = {
    "max_new_tokens": 512,
    "top_p": 0.9,
    "temperature": 1,
}


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        payload = {
            "inputs": [[
                {"role": "user", "content": prompt},
            ]],
        }
        
        input_str = json.dumps(payload)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes):
        response_json = json.loads(output.read().decode("utf-8"))
        return (f"> {response_json[0]['generation']['role'].capitalize()}: {response_json[0]['generation']['content']}")

In [4]:
from langchain import SagemakerEndpoint

content_handler = ContentHandler()

llm=SagemakerEndpoint(
        endpoint_name="meta-textgeneration-llama-2-13b-f-2023-07-30-08-44-18-636",
        region_name="us-east-1",
        #model_kwargs={"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6},
        content_handler=content_handler,
        model_kwargs=parameters,
        endpoint_kwargs = {"CustomAttributes": "accept_eula=true"}
    )

In [5]:
from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

import os
import json

os.environ["SERPAPI_API_KEY"]=open("serpapi-key").read()

llm = llm
search = SerpAPIWrapper()

tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
self_ask_with_search.run(
    "How high was Amazon's share price in 2013? "
)



> Entering new AgentExecutor chain...


OutputParserException: Could not parse output: > Assistant:  Yes, follow-up questions are needed.

Please provide the follow-up questions.